### 将csv文件根据cause进行分割

In [3]:
import pandas as pd
from pathlib import Path

file_path='./added_CAMS_data.csv'
# Load the CSV file
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')  # 'latin1' is another common encoding
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='cp1252')  # Another common encoding in Windows


# Ensure the output directory exists
output_dir = Path('./added_output/')
output_dir.mkdir(parents=True, exist_ok=True)

# Group the dataframe by the 'ANNOTATIONS' column
grouped = df.groupby('cause')

# Iterate over each group and save to a separate CSV file
for name, group in grouped:
    output_file_path = output_dir / f'{name}.csv'
    group.to_csv(output_file_path, index=False)

print("Files have been saved successfully.")


Files have been saved successfully.


### 翻译csv文件并导出为中文csv文件

In [2]:
import csv
from googletrans import Translator
import pandas as pd

translator = Translator()

def translate_to_chinese(text):
    try:
        # 尝试翻译文本
        translation = translator.translate(text, dest="zh-cn")
        return translation.text
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return text  # 如果翻译失败，返回原始文本


# 读取CSV文件
df = pd.read_csv('./added_output/0.csv')

# 对第一列进行翻译
df.iloc[:, 0] = df.iloc[:, 0].apply(translate_to_chinese)

# 保存修改后的CSV文件
df.to_csv('translated_output_file.csv', index=False)

print("Translation completed and file saved.")


An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attribute 'raise_Exception'
An error occurred during translation: 'Translator' object has no attri

### 删选一些单词少的event


In [4]:
from langchain.llms import Ollama
from guidance import models, gen
import json
import os
import pandas as pd
from tqdm import tqdm


input_filename='0_test_event.json'
output_filename = "0_test_event_filtered_data.json"


with open(input_filename, 'r') as file:
        try:
            data = json.load(file)
        except:
            print("error")

filtered_data_list = [element for element in data if len(element["Event"].split()) >= 5]

# 保存为新的 JSON 文件

with open(output_filename, 'w', encoding='utf-8') as json_file:
    json.dump(filtered_data_list, json_file, indent=4)

print(f"Before:{len(data)}")
print(f"After:{len(filtered_data_list)}")
print(f"已将过滤后的数据保存到 '{output_filename}' 文件中。")

Before:50
After:40
已将过滤后的数据保存到 '0_test_event_filtered_data.json' 文件中。


In [4]:
import json
import glob
import os

# 定义原始文件夹和目标文件夹
source_folder = '/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B'
target_folder = '/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B_modified'

# 确保目标文件夹存在
os.makedirs(target_folder, exist_ok=True)

# 获取所有json文件的列表
files = glob.glob(os.path.join(source_folder, '*.json'))

# 初始化ID
current_id = 1

for file in files:
    # 读取文件内容
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 为每个Post添加ID
    for item in data:
        item['ID'] = current_id
        current_id += 1
    
    # 写回修改后的数据到新文件夹中
    # 获取原始文件名
    base_name = os.path.basename(file)
    new_file_name = os.path.join(target_folder, f"modified_{base_name}")
    with open(new_file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(file)

print("所有文件处理完毕。")


/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/3_2_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/5_2_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/4_8_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/5_0_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/5_1_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/3_7_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/4_3_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/4_2_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EAS_fewshot_70B/5_9_event_filtered_IAS_IAS2EAS.json
/home/qiang/projects/CAMS/CAMS/data/added_output/IAS_EA

### 给生成得json增加ID，可以在出错误后继续生成

In [ ]:
import os
import re

# 定义含有JSON文件的文件夹路径
folder_path = '/path/to/your/json/files'

# 获取文件夹中所有JSON文件的列表
json_files = os.listdir(folder_path)

# 使用正则表达式匹配文件名中的数字，并转换为整型
def get_file_number(filename):
    match = re.match(r"(\d+)_.*\.json$", filename)
    return int(match.group(1)) if match else -1

# 对文件进行排序
sorted_files = sorted(json_files, key=get_file_number)

# 遍历排序后的文件
for file in sorted_files:
    file_path = os.path.join(folder_path, file)
    # 此处可以添加你处理每个文件的逻辑
    print(f"Processing file: {file_path}")

# 输出完成信息
print("文件遍历完成。")


In [7]:
import os
import glob
import re

# 定义含有JSON文件的文件夹路径
folder_path = '/home/qiang/projects/CAMS/CAMS/data/added_output/SAS_UAS/SAS_fewshot_70B'

# 获取文件夹中所有JSON文件的列表
json_files = os.listdir(folder_path)

# 使用正则表达式匹配文件名中的数字，并转换为整型
def get_file_number(filename):
    match = re.match(r"(\d+)_.*\.json$", filename)
    return int(match.group(1)) if match else -1

# 对文件进行排序
sorted_files = sorted(json_files, key=get_file_number)
print(sorted_files)
# 初始化ID
current_id = 1

# # 遍历并重命名每个文件
for file in sorted_files:
    file_path=os.path.join(folder_path,file)
    # 构建新文件名，格式为：ID_originalname.json
    new_file_name = f"{current_id}_{os.path.basename(file_path)}"
    new_file_path = os.path.join(folder_path, new_file_name)
    print(new_file_path)
    # 重命名文件
    os.rename(file_path, new_file_path)
    
    # 更新ID
    current_id += 1
    
print("所有JSON文件重命名完成。")


['0_0_event_filtered_SAS.json', '0_4_event_filtered_SAS.json', '0_3_event_filtered_SAS.json', '0_5_event_filtered_SAS.json', '0_2_event_filtered_SAS.json', '0_1_event_filtered_SAS.json', '1_0_event_filtered_SAS.json', '1_1_event_filtered_SAS.json', '1_2_event_filtered_SAS.json', '2_5_event_filtered_SAS.json', '2_0_event_filtered_SAS.json', '2_1_event_filtered_SAS.json', '2_6_event_filtered_SAS.json', '2_7_event_filtered_SAS.json', '2_2_event_filtered_SAS.json', '2_4_event_filtered_SAS.json', '2_3_event_filtered_SAS.json', '3_8_event_filtered_SAS.json', '3_5_event_filtered_SAS.json', '3_3_event_filtered_SAS.json', '3_7_event_filtered_SAS.json', '3_2_event_filtered_SAS.json', '3_1_event_filtered_SAS.json', '3_6_event_filtered_SAS.json', '3_4_event_filtered_SAS.json', '3_0_event_filtered_SAS.json', '4_17_event_filtered_SAS.json', '4_19_event_filtered_SAS.json', '4_0_event_filtered_SAS.json', '4_6_event_filtered_SAS.json', '4_3_event_filtered_SAS.json', '4_1_event_filtered_SAS.json', '4_13

In [7]:
import json
import os

# 指定文件夹路径
folder_path = '/home/qiang/projects/CAMS/CAMS/data/added_output/SAS_UAS/SAS_fewshot_70B'

# 存储超过8个换行符的post及其位置
posts_with_too_many_newlines = []

# 遍历文件夹中的所有JSON文件
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            for item in data:
                post = item['Post']
                num_newlines = post.count('\n')
                if num_newlines > 8:
                    posts_with_too_many_newlines.append({
                        'File': filename,
                        'Index': data.index(item),
                        'Number of Newlines': num_newlines
                    })

# 打印结果
for post_info in posts_with_too_many_newlines:
    print(f"File: {post_info['File']}, Index: {post_info['Index']}, Number of Newlines: {post_info['Number of Newlines']}")


File: 17_2_3_event_filtered_SAS.json, Index: 14, Number of Newlines: 12
File: 6_0_1_event_filtered_SAS.json, Index: 3, Number of Newlines: 10
File: 25_3_4_event_filtered_SAS.json, Index: 27, Number of Newlines: 10
File: 7_1_0_event_filtered_SAS.json, Index: 39, Number of Newlines: 10
File: 48_5_16_event_filtered_SAS.json, Index: 17, Number of Newlines: 10
File: 20_3_3_event_filtered_SAS.json, Index: 38, Number of Newlines: 10
File: 54_5_8_event_filtered_SAS.json, Index: 8, Number of Newlines: 10


In [6]:
def remove_posts_with_excessive_newlines(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r+') as file:
                data = json.load(file)
                for i, entry in enumerate(data):
                    if entry.get('Post', '').count('\n') > 50:
                        del data[i]
                file.seek(0)
                json.dump(data, file, indent=4)
                file.truncate()

# 调用函数并传入包含JSON文件的文件夹路径
remove_posts_with_excessive_newlines('/home/qiang/projects/CAMS/CAMS/data/added_output/SAS_UAS/SAS_fewshot_70B')